In [ ]:
import pandas as pd       
import os 
import math 
import numpy as np
import matplotlib.pyplot as plt  
import IPython.display as ipd  # To play sound in the notebook
import librosa
import librosa.display
import os
from __future__ import print_function


import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter


from six.moves import xrange

import umap

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from torchsummary import summary
import random

In [81]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [82]:
# PATHS 

raw_data = '/home/ubuntu/voice_conversion/data/raw/VCTK-Corpus'
interim_data = os.path.join('/home/ubuntu/voice_conversion/data', 'interim')
spectogram_array_path =  os.path.join(interim_data, 'spectogram_array')
spectrogram_path =os.path.join(interim_data, 'spectogram') 
audio_path = '/home/ubuntu/voice_conversion/data/raw/VCTK-Corpus/wav48/'

In [7]:
# MKDIR
os.mkdir('/home/ubuntu/voice_conversion/data/interim')

FileExistsError: [Errno 17] File exists: '/home/ubuntu/voice_conversion/data/interim'

In [21]:
os.mkdir(spectogram_array_path)

## Load Data 

In [83]:
import os
import glob

files_np = list(glob.glob(os.path.join(spectogram_array_path,'*.*')))

In [84]:
X_list = []
Y_list = []
for file in files_np[:100]: 
    trial_x = np.load(file)
    trial_x =torch.tensor(trial_x, device=device).float()
    trial_y = file.split('spectogram_array/')[1].split('_')[0][-3:]
    trial_y =torch.tensor(int(trial_y), device=device).float()
    X_list.append(trial_x)
    Y_list.append(trial_y)


In [85]:
nX_list = []
col_shape = []
for file in files_np[:100]: 
    trial_x = np.load(file)
    _, cols = trial_x.shape
    nX_list.append(trial_x)
    col_shape.append(cols)
    

In [86]:
max(col_shape)

326

In [87]:
for i,a in enumerate(nX_list):
  rows, cols = a.shape
  #print(rows, cols)
  if cols != 326:
    nX_list[i] = np.hstack([a, np.zeros(( rows, 326 - cols), dtype=a.dtype)])

In [88]:
X_list = []
for file in nX_list: 
    trial_x =torch.tensor(file, device=device).float()
    X_list.append(trial_x)

In [89]:
rows, cols = nX_list[0].shape

In [90]:
x_Tensor= torch.stack(X_list)

In [32]:
y_Tensor= torch.stack(Y_list)

In [33]:
training_tensordataset = torch.utils.data.TensorDataset(x_Tensor, y_Tensor)

In [34]:
training_loader = torch.utils.data.DataLoader(training_tensordataset, batch_size= 10 , shuffle=True)

## Vector Quantizer Layer

This layer takes a tensor to be quantized. The channel dimension will be used as the space in which to quantize. All other dimensions will be flattened and will be seen as different examples to quantize.

The output tensor will have the same shape as the input.

As an example for a `BCHW` tensor of shape `[16, 64, 32, 32]`, we will first convert it to an `BHWC` tensor of shape `[16, 32, 32, 64]` and then reshape it into `[16384, 64]` and all `16384` vectors of size `64`  will be quantized independently. In otherwords, the channels are used as the space in which to quantize. All other dimensions will be flattened and be seen as different examples to quantize, `16384` in this case.

In [94]:
batch_size = 10
num_training_updates = 1500

# Encoder
num_hiddens = 768
num_residual_hiddens = 32
num_residual_layers = 2
# input_dim: 256

# VQ
# This value is not that important, usually 64 works.
# This will not change the capacity in the information-bottleneck.
embedding_dim = 64
# The higher this value, the higher the capacity in the information bottleneck.
num_embeddings = 29

commitment_cost = 0.25

decay = -0.99

learning_rate = 1e-3

In [92]:
class VectorQuantizer(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, commitment_cost, device):
        super(VectorQuantizer, self).__init__()
        
        self._embedding_dim = embedding_dim
        self._num_embeddings = num_embeddings
        
        self._embedding = nn.Embedding(self._num_embeddings, self._embedding_dim)
        self._embedding.weight.data.uniform_(-1/self._num_embeddings, 1/self._num_embeddings)
        self._commitment_cost = commitment_cost
        self._device = device

    def forward(self, inputs):
        
        # convert inputs from BCHW -> BHWC
        inputs = inputs.permute(1, 2, 0).contiguous()
        input_shape = inputs.shape
        
        print('shape of inputs in VectorQuantizer.forward',inputs.shape)
        _, time, batch_size = input_shape
        
        # Flatten input
        flat_input = inputs.view(-1, self._embedding_dim)
        print('shape of flat_input in VectorQuantizer.forward',flat_input.size())
        print('device of flat_input',flat_input.device)
        
        # Calculate distances
        distances = (torch.sum(flat_input**2, dim=1, keepdim=True) 
                    + torch.sum(self._embedding.weight**2, dim=1)
                    - 2 * torch.matmul(flat_input, self._embedding.weight.t()))
            
        # Encoding
        encoding_indices = torch.argmin(distances, dim=1).unsqueeze(1)
        encodings = torch.zeros(encoding_indices.shape[0], self._num_embeddings, device=inputs.device)
        encodings.scatter_(1, encoding_indices, 1)
        
        # Quantize and unflatten
        quantized = torch.matmul(encodings, self._embedding.weight).view(input_shape)
        print('shape of quantized in VectorQuantizer.forward',quantized.size())
        
        # Loss
        e_latent_loss = F.mse_loss(quantized.detach(), inputs)
        q_latent_loss = F.mse_loss(quantized, inputs.detach())
        loss = q_latent_loss + self._commitment_cost * e_latent_loss
        
        quantized = inputs + (quantized - inputs).detach()
        avg_probs = torch.mean(encodings, dim=0)
        perplexity = torch.exp(-torch.sum(avg_probs * torch.log(avg_probs + 1e-10)))
        #print("with inputs quantized: "quantized.size())
        
        # convert quantized from BHWC -> BCHW
        return loss, quantized.permute(2, 0, 1).contiguous(), perplexity, encodings

We will also implement a slightly modified version  which will use exponential moving averages to update the embedding vectors instead of an auxillary loss. This has the advantage that the embedding updates are independent of the choice of optimizer for the encoder, decoder and other parts of the architecture. For most experiments the EMA version trains faster than the non-EMA version.

In [19]:
class VectorQuantizerEMA(nn.Module):
    """
    Inspired from Sonnet implementation of VQ-VAE https://arxiv.org/abs/1711.00937,
    in https://github.com/deepmind/sonnet/blob/master/sonnet/python/modules/nets/vqvae.py and
    pytorch implementation of it from zalandoresearch in https://github.com/zalandoresearch/pytorch-vq-vae/blob/master/vq-vae.ipynb.
    Implements a slightly modified version of the algorithm presented in
    'Neural Discrete Representation Learning' by van den Oord et al.
    https://arxiv.org/abs/1711.00937
    The difference between VectorQuantizerEMA and VectorQuantizer is that
    this module uses exponential moving averages to update the embedding vectors
    instead of an auxiliary loss. This has the advantage that the embedding
    updates are independent of the choice of optimizer (SGD, RMSProp, Adam, K-Fac,
    ...) used for the encoder, decoder and other parts of the architecture. For
    most experiments the EMA version trains faster than the non-EMA version.
    Input any tensor to be quantized. Last dimension will be used as space in
    which to quantize. All other dimensions will be flattened and will be seen
    as different examples to quantize.
    The output tensor will have the same shape as the input.
    For example a tensor with shape [16, 32, 32, 64] will be reshaped into
    [16384, 64] and all 16384 vectors (each of 64 dimensions)  will be quantized
    independently.
    Args:
        embedding_dim: integer representing the dimensionality of the tensors in the
            quantized space. Inputs to the modules must be in this format as well.
        num_embeddings: integer, the number of vectors in the quantized space.
            commitment_cost: scalar which controls the weighting of the loss terms (see
            equation 4 in the paper).
        decay: float, decay for the moving averages.
        epsilon: small float constant to avoid numerical instability.
    """
        
        
    def __init__(self, num_embeddings, embedding_dim, commitment_cost, decay, device, epsilon=1e-5):
           
        super(VectorQuantizerEMA, self).__init__()
        
        self._embedding_dim = embedding_dim
        self._num_embeddings = num_embeddings
        
        self._embedding = nn.Embedding(self._num_embeddings, self._embedding_dim)
        self._embedding.weight.data.normal_()
        self._commitment_cost = commitment_cost
        
        self.register_buffer('_ema_cluster_size', torch.zeros(num_embeddings))
        self._ema_w = nn.Parameter(torch.Tensor(num_embeddings, self._embedding_dim))
        self._ema_w.data.normal_()
        
        self._decay = decay
        self._epsilon = epsilon
        self._device = device

    def forward(self, inputs, compute_distances_if_possible=True, record_codebook_stats=False):
        """
        Connects the module to some inputs.
        Args:
            inputs: Tensor, final dimension must be equal to embedding_dim. All other
                leading dimensions will be flattened and treated as a large batch.
        
        Returns:
            loss: Tensor containing the loss to optimize.
            quantize: Tensor containing the quantized version of the input.
            perplexity: Tensor containing the perplexity of the encodings.
            encodings: Tensor containing the discrete encodings, ie which element
                of the quantized space each input element was mapped to.
            distances
        """

        
        # convert inputs from BCHW -> BHWC
        inputs = inputs.permute(1, 2, 0).contiguous()
        input_shape = inputs.shape
        
        # Flatten input
        flat_input = inputs.view(-1, self._embedding_dim)
        
        # Calculate distances
        distances = (torch.sum(flat_input**2, dim=1, keepdim=True) 
                    + torch.sum(self._embedding.weight**2, dim=1)
                    - 2 * torch.matmul(flat_input, self._embedding.weight.t()))
        """
        encoding_indices: Tensor containing the discrete encoding indices, ie
        which element of the quantized space each input element was mapped to.
        """
        
        # Encoding
        encoding_indices = torch.argmin(distances, dim=1).unsqueeze(1)
        encodings = torch.zeros(encoding_indices.shape[0], self._num_embeddings, dtype=torch.float).to(self._device)
        encodings.scatter_(1, encoding_indices, 1)
        
        # Compute distances between encoding vectors
        if not self.training and compute_distances_if_possible:
            _encoding_distances = [torch.dist(items[0], items[1], 2).to(self._device) for items in combinations(flat_input, r=2)]
            encoding_distances = torch.tensor(_encoding_distances).to(self._device).view(batch_size, -1)
        else:
            encoding_distances = None

        # Compute distances between embedding vectors
        if not self.training and compute_distances_if_possible:
            _embedding_distances = [torch.dist(items[0], items[1], 2).to(self._device) for items in combinations(self._embedding.weight, r=2)]
            embedding_distances = torch.tensor(_embedding_distances).to(self._device)
        else:
            embedding_distances = None

        # Sample nearest embedding
        if not self.training and compute_distances_if_possible:
            _frames_vs_embedding_distances = [torch.dist(items[0], items[1], 2).to(self._device) for items in product(flat_input, self._embedding.weight.detach())]
            frames_vs_embedding_distances = torch.tensor(_frames_vs_embedding_distances).to(self._device).view(batch_size, time, -1)
        else:
            frames_vs_embedding_distances = None
        
        
        

        
        # Use EMA to update the embedding vectors
        if self.training:
            self._ema_cluster_size = self._ema_cluster_size * self._decay + \
                                     (1 - self._decay) * torch.sum(encodings, 0)
            
            # Laplace smoothing of the cluster size
            n = torch.sum(self._ema_cluster_size.data)
            self._ema_cluster_size = (
                (self._ema_cluster_size + self._epsilon)
                / (n + self._num_embeddings * self._epsilon) * n)
            
            dw = torch.matmul(encodings.t(), flat_input)
            self._ema_w = nn.Parameter(self._ema_w * self._decay + (1 - self._decay) * dw)
            
            self._embedding.weight = nn.Parameter(self._ema_w / self._ema_cluster_size.unsqueeze(1))
        
        # Quantize and unflatten
        quantized = torch.matmul(encodings, self._embedding.weight).view(input_shape)
        
        concatenated_quantized = self._embedding.weight[torch.argmin(distances, dim=1).detach().cpu()] if not self.training or record_codebook_stats else None
        
        # Loss
        e_latent_loss = torch.mean((quantized.detach() - inputs)**2)
        loss = self._commitment_cost * e_latent_loss
        
        # Straight Through Estimator
        quantized = inputs + (quantized - inputs).detach()
        avg_probs = torch.mean(encodings, dim=0)
        """
        The perplexity a useful value to track during training.
        It indicates how many codes are 'active' on average.
        """
        perplexity = torch.exp(-torch.sum(avg_probs * torch.log(avg_probs + 1e-10)))
        
        # convert quantized from BHWC -> BCHW
        return loss, quantized.permute(2, 0, 1).contiguous(), perplexity, encodings

## Encoder & Decoder Architecture

The encoder and decoder architecture is based on a ResNet and is implemented below:

In [35]:
class Encoder(nn.Module):
    def __init__(self, in_channels, num_hiddens, num_residual_layers, num_residual_hiddens):
        super(Encoder, self).__init__()
        
        """
        2 preprocessing convolution layers with filter length 3
        and residual connections.
        """
        

        self._conv_1 = nn.Conv1d(in_channels= 1025, #??features_filters,
                                 out_channels=num_hiddens,
                                 kernel_size=3,
                                 padding=1)
        
        self._conv_2 = nn.Conv1d(in_channels=num_hiddens,
                                 out_channels=num_hiddens,
                                 kernel_size=3,
                                 padding=1)
        """
        1 strided convolution length reduction layer with filter
        length 4 and stride 2 (downsampling the signal by a factor
        of two).
        """
        
        self._conv_3 = nn.Conv1d(in_channels=num_hiddens,
                                 out_channels=num_hiddens,
                                 kernel_size=4,
                                 stride=1, 
                                 padding=2)
        """
        2 convolutional layers with length 3 and
        residual connections.
        """
        self._conv_4 = nn.Conv1d(in_channels=num_hiddens,
                                 out_channels=num_hiddens,
                                 kernel_size=3,
                                 padding=1)
        
        self._conv_5 = nn.Conv1d(in_channels=num_hiddens,
                                 out_channels=num_hiddens,
                                 kernel_size=3,
                                 padding=1)
        
        self._residual_stack = ResidualStack(in_channels=num_hiddens,
                                             num_hiddens=num_hiddens,
                                             num_residual_layers=num_residual_layers,
                                             num_residual_hiddens=num_residual_hiddens)

    def forward(self, inputs):
        print('shape of inputs in Encoder.forward',inputs.size())

        x_conv_1 = F.relu(self._conv_1(inputs))
        print('shape of x in Encoder.forward._conv_1',x_conv_1.size())

        x = F.relu(self._conv_2(x_conv_1)) + x_conv_1
        print('shape of x in Encoder.forward.relu_2',x.size())
        
        x_conv_3 = F.relu(self._conv_3(x))
        print('shape of x in Encoder.forward._conv_3',x.size())
        
        x_conv_4 = F.relu(self._conv_4(x_conv_3)) + x_conv_3
        
        x_conv_5 = F.relu(self._conv_5(x_conv_4)) + x_conv_4
        
        x = self._residual_stack(x_conv_5) + x_conv_5
        
        return x

In [21]:
class Residual(nn.Module):
    def __init__(self, in_channels, num_hiddens, num_residual_hiddens):
        
        super(Residual, self).__init__()
        
        self._block = nn.Sequential(
            
            nn.ReLU(True),
            
            nn.Conv1d(
                     in_channels=in_channels,
                      out_channels=num_residual_hiddens,
                      kernel_size=3, 
                      stride=1, 
                      padding=1, 
                      bias=False),
            
            nn.ReLU(True),
            
            nn.Conv1d(in_channels=num_residual_hiddens,
                      out_channels=num_hiddens,
                      kernel_size=1, 
                      stride=1, 
                      bias=False)
        )
    
    def forward(self, x):
        print('shape of x in Residual.forward',x.size())
        return x + self._block(x)


class ResidualStack(nn.Module):
    def __init__(self, in_channels, num_hiddens, num_residual_layers, num_residual_hiddens):
        super(ResidualStack, self).__init__()
        
        self._num_residual_layers = num_residual_layers
        self._layers = nn.ModuleList([Residual(in_channels, num_hiddens, num_residual_hiddens)
                             for _ in range(self._num_residual_layers)])

    def forward(self, x):
        print('shape of x in ResidualStack.forward',x.size())
        for i in range(self._num_residual_layers):
            x = self._layers[i](x)
            print(f'Iteration {i} shape of x in ResidualStack.forward {x.size()} ',)
        return F.relu(x)

In [51]:
class Decoder(nn.Module):
    def __init__(self, in_channels, num_hiddens, num_residual_layers, num_residual_hiddens):
        super(Decoder, self).__init__()
        
        self._conv_1 = nn.Conv1d(in_channels=in_channels,
                                 out_channels=num_hiddens,
                                 kernel_size=3,
                                 padding=1)
        
        self._upsample = nn.Upsample(scale_factor=2)
        
        self._residual_stack = ResidualStack(in_channels=num_hiddens,
                                             num_hiddens=num_hiddens,
                                             num_residual_layers=num_residual_layers,
                                             num_residual_hiddens=num_residual_hiddens)
        
        self._conv_trans_1 = nn.ConvTranspose1d(in_channels=num_hiddens, 
                                                out_channels=num_hiddens,
                                                kernel_size=3, 
                                                padding=1)
        
        self._conv_trans_2 = nn.ConvTranspose1d(in_channels=num_hiddens, 
                                                out_channels=num_hiddens,
                                                kernel_size=3,  
                                                padding=0)
        
        self._conv_trans_3 = nn.ConvTranspose1d(in_channels=num_hiddens, 
                                                out_channels=3,#out_channels
                                                kernel_size=2, 
                                                padding=0)
        

    def forward(self, inputs):
        x = self._conv_1(inputs)
        
        x = self._upsample(x)
        
        x = self._residual_stack(x)
        
        x = self._conv_trans_1(x)
        
        x = F.relu(self._conv_trans_1(x))
        
        x = F.relu(self._conv_trans_2(x))
        
        x = self._conv_trans_3(x)
        
        return x

## Train

We use the hyperparameters from the author's code:

In [40]:
#training_loader = DataLoader(training_data, 
#                             batch_size=batch_size, 
#                             shuffle=True,
#                             pin_memory=True)

In [41]:
#validation_loader = DataLoader(validation_data,
#                               batch_size=32,
#                               shuffle=True,
#                               pin_memory=True)

In [37]:
class Model(nn.Module):
    def __init__(self, num_hiddens, num_residual_layers, num_residual_hiddens, 
                 num_embeddings, embedding_dim, commitment_cost, decay=0):
        super(Model, self).__init__()
        
        self._encoder = Encoder(1, num_hiddens,
                                num_residual_layers, 
                                num_residual_hiddens)
        self._pre_vq_conv = nn.Conv1d(in_channels=num_hiddens, 
                                      out_channels=embedding_dim,
                                      kernel_size=1, 
                                      stride=1)
        if decay > 0.0:
            self._vq_vae = VectorQuantizerEMA(num_embeddings, embedding_dim, 
                                              commitment_cost, decay)
        else:
            self._vq_vae = VectorQuantizer(num_embeddings, embedding_dim,
                                           commitment_cost, device)
        self._decoder = Decoder(embedding_dim,
                                num_hiddens, 
                                num_residual_layers, 
                                num_residual_hiddens)

    def forward(self, x):
        print("Model::forward")
        #print(x.size())
        z = self._encoder(x)
        z = self._pre_vq_conv(z)
        loss, quantized, perplexity, _ = self._vq_vae(z)
        #print("quantized ")
        #print( quantized.size())
        x_recon = self._decoder(quantized)

        return loss, x_recon, perplexity

In [114]:
model = Model(num_hiddens, num_residual_layers, num_residual_hiddens,
              num_embeddings, embedding_dim, 
              commitment_cost, decay).to(device)

In [122]:
summary(model, (1025, 326))

Model::forward
shape of inputs in Encoder.forward torch.Size([2, 1025, 326])
shape of x in Encoder.forward._conv_1 torch.Size([2, 768, 326])
shape of x in Encoder.forward.relu_2 torch.Size([2, 768, 326])
shape of x in Encoder.forward._conv_3 torch.Size([2, 768, 326])
shape of x in ResidualStack.forward torch.Size([2, 768, 327])
shape of x in Residual.forward torch.Size([2, 768, 327])
Iteration 0 shape of x in ResidualStack.forward torch.Size([2, 768, 327]) 
shape of x in Residual.forward torch.Size([2, 768, 327])
Iteration 1 shape of x in ResidualStack.forward torch.Size([2, 768, 327]) 
shape of inputs in VectorQuantizer.forward torch.Size([64, 327, 2])
shape of flat_input in VectorQuantizer.forward torch.Size([654, 64])
device of flat_input cuda:0
shape of quantized in VectorQuantizer.forward torch.Size([64, 327, 2])
shape of x in ResidualStack.forward torch.Size([2, 768, 654])
shape of x in Residual.forward torch.Size([2, 768, 654])
Iteration 0 shape of x in ResidualStack.forward tor

TypeError: can't multiply sequence by non-int of type 'list'

In [39]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate, amsgrad=False)

In [40]:
for i in training_loader:
    train = i
    break

In [41]:
np.shape(train)

(2,)

In [42]:
train[0].size()

torch.Size([10, 1025, 326])

In [43]:
train[1].size()

torch.Size([10])

In [44]:
enc = Encoder(1, num_hiddens,
                                num_residual_layers, 
                                num_residual_hiddens).to(device)

In [45]:
enc._conv_1(train[0])

tensor([[[-4.9948e+00, -5.9375e+00, -5.9967e+00,  ...,  1.7835e-02,
           1.7835e-02,  1.7835e-02],
         [-1.3668e+01, -6.0679e+00, -7.2850e+00,  ...,  1.2581e-02,
           1.2581e-02,  1.2581e-02],
         [-1.0343e+01, -1.0765e+01, -1.0108e+01,  ..., -1.7396e-02,
          -1.7396e-02, -1.7396e-02],
         ...,
         [ 1.8608e+01,  1.1135e+01,  1.0847e+01,  ...,  7.2693e-03,
           7.2693e-03,  7.2693e-03],
         [ 2.4501e+01,  2.7265e+01,  2.7245e+01,  ...,  1.4419e-02,
           1.4419e-02,  1.4419e-02],
         [-1.1214e+01, -2.7150e+01, -2.4559e+01,  ..., -1.2557e-02,
          -1.2557e-02, -1.2557e-02]],

        [[-3.4664e+00, -4.9100e+00, -4.9295e+00,  ...,  1.7835e-02,
           1.7835e-02,  1.7835e-02],
         [-1.7095e+01, -8.5921e+00, -8.2084e+00,  ...,  1.2581e-02,
           1.2581e-02,  1.2581e-02],
         [-1.1602e+01, -1.1028e+01, -1.0709e+01,  ..., -1.7396e-02,
          -1.7396e-02, -1.7396e-02],
         ...,
         [ 1.5291e+01,  9

In [95]:
enc.forward(train[0]).shape

shape of inputs in Encoder.forward torch.Size([10, 1025, 326])
shape of x in Encoder.forward._conv_1 torch.Size([10, 768, 326])
shape of x in Encoder.forward.relu_2 torch.Size([10, 768, 326])
shape of x in Encoder.forward._conv_3 torch.Size([10, 768, 326])
shape of x in ResidualStack.forward torch.Size([10, 768, 327])
shape of x in Residual.forward torch.Size([10, 768, 327])
Iteration 0 shape of x in ResidualStack.forward torch.Size([10, 768, 327]) 
shape of x in Residual.forward torch.Size([10, 768, 327])
Iteration 1 shape of x in ResidualStack.forward torch.Size([10, 768, 327]) 


torch.Size([10, 768, 327])

In [96]:
model.forward(train[0])

Model::forward
shape of inputs in Encoder.forward torch.Size([10, 1025, 326])
shape of x in Encoder.forward._conv_1 torch.Size([10, 768, 326])
shape of x in Encoder.forward.relu_2 torch.Size([10, 768, 326])
shape of x in Encoder.forward._conv_3 torch.Size([10, 768, 326])
shape of x in ResidualStack.forward torch.Size([10, 768, 327])
shape of x in Residual.forward torch.Size([10, 768, 327])
Iteration 0 shape of x in ResidualStack.forward torch.Size([10, 768, 327]) 
shape of x in Residual.forward torch.Size([10, 768, 327])
Iteration 1 shape of x in ResidualStack.forward torch.Size([10, 768, 327]) 
shape of x in ResidualStack.forward torch.Size([10, 768, 654])
shape of x in Residual.forward torch.Size([10, 768, 654])
Iteration 0 shape of x in ResidualStack.forward torch.Size([10, 768, 654]) 
shape of x in Residual.forward torch.Size([10, 768, 654])
Iteration 1 shape of x in ResidualStack.forward torch.Size([10, 768, 654]) 


(tensor(111.5821, device='cuda:0', grad_fn=<AddBackward0>),
 tensor([[[ 2.6973e-03,  3.6762e-03, -1.4785e-03,  ..., -5.4070e-03,
            4.0411e-03,  4.1874e-03],
          [-1.5532e-02, -8.3419e-03, -1.3484e-02,  ..., -3.4968e-03,
           -5.1036e-03, -1.3369e-02],
          [-2.6497e-02, -2.9566e-02, -4.2982e-02,  ..., -3.4109e-02,
           -2.7443e-02, -2.0236e-02],
          ...,
          [-2.6597e-02, -2.4212e-02, -2.8920e-02,  ..., -3.3386e-02,
           -2.9299e-02, -1.8367e-02],
          [-1.2812e-03,  8.1935e-04,  5.5526e-03,  ..., -2.4481e-03,
           -1.1364e-05, -8.1752e-03],
          [ 9.1256e-03,  1.8751e-02,  1.9326e-02,  ...,  1.8490e-02,
            2.2438e-02,  1.6414e-02]],
 
         [[ 2.0671e-03,  4.5945e-03, -4.1662e-04,  ..., -3.0963e-03,
            4.7270e-03,  5.1625e-03],
          [-1.3824e-02, -5.0775e-03, -1.2608e-02,  ..., -5.0051e-03,
           -5.6270e-03, -1.3650e-02],
          [-2.4458e-02, -2.4846e-02, -4.0834e-02,  ..., -3.3022e-0

In [99]:
pre_vq_conv = nn.Conv1d(in_channels=num_hiddens, 
                                      out_channels=embedding_dim,
                                      kernel_size=1, 
                                      stride=1)

In [107]:
x = model._encoder(train[0])
x = model._pre_vq_conv(x)
_,x,_,_ = model._vq_vae(x)
x = model._decoder(x)

shape of inputs in Encoder.forward torch.Size([10, 1025, 326])
shape of x in Encoder.forward._conv_1 torch.Size([10, 768, 326])
shape of x in Encoder.forward.relu_2 torch.Size([10, 768, 326])
shape of x in Encoder.forward._conv_3 torch.Size([10, 768, 326])
shape of x in ResidualStack.forward torch.Size([10, 768, 327])
shape of x in Residual.forward torch.Size([10, 768, 327])
Iteration 0 shape of x in ResidualStack.forward torch.Size([10, 768, 327]) 
shape of x in Residual.forward torch.Size([10, 768, 327])
Iteration 1 shape of x in ResidualStack.forward torch.Size([10, 768, 327]) 
shape of x in ResidualStack.forward torch.Size([10, 768, 654])
shape of x in Residual.forward torch.Size([10, 768, 654])
Iteration 0 shape of x in ResidualStack.forward torch.Size([10, 768, 654]) 
shape of x in Residual.forward torch.Size([10, 768, 654])
Iteration 1 shape of x in ResidualStack.forward torch.Size([10, 768, 654]) 


In [97]:
vq_vae = VectorQuantizer(num_embeddings, embedding_dim, 
                                              commitment_cost, decay, device)

TypeError: __init__() takes 5 positional arguments but 6 were given

In [98]:
vq_vae.forward(enc.forward(train[0]))

shape of inputs in Encoder.forward torch.Size([10, 1025, 326])
shape of x in Encoder.forward._conv_1 torch.Size([10, 768, 326])
shape of x in Encoder.forward.relu_2 torch.Size([10, 768, 326])
shape of x in Encoder.forward._conv_3 torch.Size([10, 768, 326])
shape of x in ResidualStack.forward torch.Size([10, 768, 327])
shape of x in Residual.forward torch.Size([10, 768, 327])
Iteration 0 shape of x in ResidualStack.forward torch.Size([10, 768, 327]) 
shape of x in Residual.forward torch.Size([10, 768, 327])
Iteration 1 shape of x in ResidualStack.forward torch.Size([10, 768, 327]) 


RuntimeError: expected device cuda:0 but got device cpu

In [104]:
vq_vae

VectorQuantizerEMA(
  (_embedding): Embedding(29, 64)
)

In [ ]:
vq_vae = VectorQuantizerEMA(num_embeddings, embedding_dim, 
                                              commitment_cost, decay)

In [132]:
res  = ResidualStack(in_channels=num_hiddens,
                                             num_hiddens=num_hiddens,
                                             num_residual_layers=num_residual_layers,
                                             num_residual_hiddens=num_residual_hiddens)

In [69]:
res.forward(train[0])

NameError: name 'res' is not defined

In [109]:
data_recon.shape

torch.Size([10, 768, 656])

In [110]:
data.shape

torch.Size([10, 1025, 326])

In [120]:
summary(model,(768, 1025, 3))

Model::forward
shape of inputs in Encoder.forward torch.Size([2, 768, 1025, 3])


RuntimeError: Expected 3-dimensional input for 3-dimensional weight 768 1025 3, but got 4-dimensional input of size [2, 768, 1025, 3] instead

In [108]:
model.train()
train_res_recon_error = []
train_res_perplexity = []

for i in xrange(num_training_updates):
    (data, _) = next(iter(training_loader))
    data = data.to(device)
    optimizer.zero_grad()
    print("Data size")
    print(data.size())

    vq_loss, data_recon, perplexity = model(data)
    recon_error = F.mse_loss(data_recon, data) / data_variance
    loss = recon_error + vq_loss
    loss.backward()

    optimizer.step()
    
    train_res_recon_error.append(recon_error.item())
    train_res_perplexity.append(perplexity.item())

    if (i+1) % 100 == 0:
        print('%d iterations' % (i+1))
        print('recon_error: %.3f' % np.mean(train_res_recon_error[-100:]))
        print('perplexity: %.3f' % np.mean(train_res_perplexity[-100:]))
        print()

Data size
torch.Size([10, 1025, 326])
Model::forward
shape of inputs in Encoder.forward torch.Size([10, 1025, 326])
shape of x in Encoder.forward._conv_1 torch.Size([10, 768, 326])
shape of x in Encoder.forward.relu_2 torch.Size([10, 768, 326])
shape of x in Encoder.forward._conv_3 torch.Size([10, 768, 326])
shape of x in ResidualStack.forward torch.Size([10, 768, 327])
shape of x in Residual.forward torch.Size([10, 768, 327])
Iteration 0 shape of x in ResidualStack.forward torch.Size([10, 768, 327]) 
shape of x in Residual.forward torch.Size([10, 768, 327])
Iteration 1 shape of x in ResidualStack.forward torch.Size([10, 768, 327]) 
shape of x in ResidualStack.forward torch.Size([10, 768, 654])
shape of x in Residual.forward torch.Size([10, 768, 654])
Iteration 0 shape of x in ResidualStack.forward torch.Size([10, 768, 654]) 
shape of x in Residual.forward torch.Size([10, 768, 654])
Iteration 1 shape of x in ResidualStack.forward torch.Size([10, 768, 654]) 


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Using a target size (torch.Size([10, 1025, 326])) that is different to the input size (torch.Size([10, 768, 656])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  del sys.path[0]


RuntimeError: The size of tensor a (656) must match the size of tensor b (326) at non-singleton dimension 2